<a href="https://colab.research.google.com/github/binhvd/Data-Analytics-3-Labs/blob/main/10_Sentiment_Analysis_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup the Environment

In [ ]:
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, SimpleRNN, Activation, Dropout, Conv1D
from tensorflow.keras.layers import Embedding, Flatten, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

import pandas as pd
import numpy as np
import spacy
from sklearn.metrics import classification_report

## Exploratory Data Analysis

In [ ]:
!wget https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv

--2022-10-18 07:38:13--  https://raw.githubusercontent.com/haochen23/nlp-rnn-lstm-sentiment/master/training.1600000.processed.noemoticon.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2989873 (2.9M) [text/plain]
Saving to: ‘training.1600000.processed.noemoticon.csv’

training.1600000.pr 100%[===================>]   2.85M  --.-KB/s    in 0.01s   

2022-10-18 07:38:14 (248 MB/s) - ‘training.1600000.processed.noemoticon.csv’ saved [2989873/2989873]



In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", header=None, encoding='latin-1')
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
# Check for missing values
data.isnull().any()

0    False
1    False
2    False
3    False
4    False
5    False
dtype: bool

## Preparing Data

We only care about the tweet text and tweet sentiment information, which stored in the 5th column and 0th column in the dataset. In the sentiment column, 0 represents negative, and 1 represents positive.

We organize the data as data_X contains all the tweet text, data_y contains the labels.

The following code will convert the tweet text data_X to sequence format that will be feed into RNNs

In [ ]:
data_X = data[5]
print(data_X)

0        @switchfoot http://twitpic.com/2y1zl - Awww, t...
1        is upset that he can't update his Facebook by ...
2        @Kenichan I dived many times for the ball. Man...
3          my whole body feels itchy and like its on fire 
4        @nationwideclass no, it's not behaving at all....
                               ...                        
19995    Just woke up. Having no school is the best fee...
19996    TheWDB.com - Very cool to hear old Walt interv...
19997    Are you ready for your MoJo Makeover? Ask me f...
19998    Happy 38th Birthday to my boo of alll time!!! ...
19999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: 5, Length: 20000, dtype: object


#### Label:
*   0 -> NEGATIVE
*   2 -> NEUTRAL
*   4 -> POSITIVE

In [ ]:
data_y = pd.get_dummies(data[0]).to_numpy()
print(data_y)

[[1 0]
 [1 0]
 [1 0]
 ...
 [0 1]
 [0 1]
 [0 1]]


Splitting Data for Training

In [ ]:
# TODO: Split data into train and valid sets
...

## Tokenization

In [ ]:
MAX_VOCAB = 18000
MAX_LEN = 150
EMBED_SIZE = 200

In [ ]:
# TODO: Tokenize inputs
...

In [ ]:
# TODO: Text padding
...

In [ ]:
train_X

array([[  50, 3733,   37, ..., 2638,    1,  308],
       [ 178, 7110, 2291, ...,  711,  110,   39],
       [ 120,  105,  316, ...,  629,  105,   86],
       ...,
       [   8,  933,  118, ..., 1088,  383,    7],
       [ 118,   10,   34, ...,    0,    0,    0],
       [   9,    1,  750, ...,    0,    0,    0]], dtype=int32)

## Preparing Word Embeddings using the GloVe Model

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gensim.downloader as api

# Load the twitter embeddings model. This model is trained on 2 billion tweets, which contains 27 billion tokens, 1.2 million vocabs.
# might take a while
glove_model = api.load("glove-twitter-200")

[==================================================] 100.0% 758.5/758.5MB downloaded


In [ ]:
# calcultaete number of words
nb_words = len(word_index) + 1
print('All words: ', nb_words)

# obtain the word embedding matrix
embedding_matrix = np.zeros((nb_words, EMBED_SIZE))
for word, i in word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 4103


**Explanation of the steps performed till now**

Tweets: Is upset that he can't update his Facebook..

Expected Input to RNN model - 
Is - Embeddings [200] (32)

upset - Embeddings [200] (450)

that - Embeddings [200] (43)

he - Embeddings [200] (56)

1. Vocabulary of all tweets: 30257 unique tokens
2. Unique token IDs: ID (1, 2, 3, 4... for all the 30257 tokens)
3. Tweets represented as the sequence of IDs [32 450 43 56 ...]

Padding: 
"Commonly in RNN's, we take the final output or hidden state and use this to make a prediction (or do whatever task we are trying to do).
If we send a bunch of 0's to the RNN before taking the final output (i.e. 'post' padding as you describe), then the hidden state of the network at the final word in the sentence would likely get 'flushed out' to some extent by all the zero inputs that come after this word.
So intuitively, this might be why pre-padding is more popular/effective." - [link](https://stackoverflow.com/questions/46298793/how-does-choosing-between-pre-and-post-zero-padding-of-sequences-impact-results)

Padding for RNNs - [Link](https://datascience.stackexchange.com/questions/49168/padding-sequences-for-neural-sequence-models-rnns)

[Paper](https://arxiv.org/abs/1903.07288)





## Training and Evaluation


Train and evaluate the SimpleRNN, LSTM, and GRU networks on our prepared dataset.

We are using the pre-trained word embeddings from the glove.twitter.27B.200d.txt data. Using the pre-trained word embeddings as weights for the Embedding layer leads to better results and faster convergence.

We set each models to run 20 epochs, but we also set EarlyStopping rules to prevent overfitting. The results of the SimpleRNN, LSTM, GRU models can be seen below.

In [ ]:
model_rnn = Sequential()
model_rnn.add(Embedding(nb_words, EMBED_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable = False))

# TODO: Add a SimpleRNN layer
...

model_rnn.add(Dense(2, activation='softmax'))
model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model_rnn.fit(train_X, train_y, epochs=20, batch_size=120,
          validation_data=(valid_X, valid_y), callbacks=EarlyStopping(monitor='val_accuracy', mode='max',patience=3))

predictions = model_rnn.predict(valid_X)
predictions = predictions.argmax(axis=1)
print(classification_report(valid_y.argmax(axis=1), predictions))

Epoch 1/20
134/134 [==============================] - 4s 23ms/step - loss: 0.6031 - accuracy: 0.6779 - val_loss: 0.5610 - val_accuracy: 0.7125
Epoch 2/20
134/134 [==============================] - 3s 20ms/step - loss: 0.5531 - accuracy: 0.7225 - val_loss: 0.5515 - val_accuracy: 0.7237
Epoch 3/20
134/134 [==============================] - 3s 20ms/step - loss: 0.5360 - accuracy: 0.7319 - val_loss: 0.5487 - val_accuracy: 0.7215
Epoch 4/20
134/134 [==============================] - 3s 20ms/step - loss: 0.5275 - accuracy: 0.7381 - val_loss: 0.5512 - val_accuracy: 0.7185
Epoch 5/20
125/125 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.72      0.70      0.71      2018
           1       0.71      0.73      0.72      1982

    accuracy                           0.72      4000
   macro avg       0.72      0.72      0.72      4000
weighted avg       0.72      0.72      0.72      4000



## LSTM and GRUs

In [ ]:
# TODO: Train a LSTM model by replacing the SimpleRNN layer with a LSTM layer
model_lstm = ...

# TODO: Print a classification report for the model
...

In [ ]:
# TODO: Train a GRU model by replacing the SimpleRNN layer with a GRU layer
model_gru = ...


# TODO: Print a classification report for the model
...

## Evaluation

In [ ]:
import time

def predict(model, text):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=MAX_LEN)
    # Predict
    score = model.predict([x_test])[0]

    return {"NEGATIVE": score[0], "POSITIVE": score[1],
       "elapsed_time": time.time()-start_at}  

In [ ]:
# TODO: Try few sentences to check the models
predict(model_lstm, "I feel not so good today")

1/1 [==============================] - 0s 27ms/step


{'NEGATIVE': 0.60516524,
 'POSITIVE': 0.39483473,
 'elapsed_time': 0.09723186492919922}

## Pre-trained Word Embeddings

Try training the RNNs with word embeddings but without the pre-trained weight and compare the results with the pre-trained model.


In [ ]:
# TODO: Remove embedding_matrix and set trainable=TRUE
...